In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [ ]:
df = pd.read_csv("diabetes.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# Check for missing values
df.isnull().sum()

In [ ]:
# Columns where zero is invalid and should be replaced
zero_invalid_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
# Replace zeros with median values in these columns
for col in zero_invalid_cols:
    median_val = df[col].median()
    df[col] = df[col].replace(0, median_val)


In [ ]:
# Split features and target
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Train-test split (80-20)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Define parameters to experiment
k_values = [3, 5, 7]
distance_metrics = ['euclidean', 'manhattan', 'minkowski', 'chebyshev']
minkowski_p = 3  # for Minkowski distance

# Dictionary to store results
knn_results = {}

for metric in distance_metrics:
    for k in k_values:
        if metric == 'minkowski':
            knn = KNeighborsClassifier(n_neighbors=k, metric=metric, p=minkowski_p)
        else:
            knn = KNeighborsClassifier(n_neighbors=k, metric=metric)
        
        knn.fit(X_train_scaled, y_train)
        y_pred = knn.predict(X_test_scaled)
        
        # Store metrics
        knn_results[(metric, k)] = {
            "accuracy": accuracy_score(y_test, y_pred),
            "error_rate": 1 - accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred),
            "recall": recall_score(y_test, y_pred),
            "f1_score": f1_score(y_test, y_pred),
            "confusion_matrix": confusion_matrix(y_test, y_pred)
        }
        
        # Print metrics
        print(f"\nKNN with k={k}, metric={metric}")
        print(f"Accuracy: {knn_results[(metric, k)]['accuracy']:.3f}")
        print(f"Error Rate: {knn_results[(metric, k)]['error_rate']:.3f}")
        print(f"Precision: {knn_results[(metric, k)]['precision']:.3f}")
        print(f"Recall: {knn_results[(metric, k)]['recall']:.3f}")
        print(f"F1-score: {knn_results[(metric, k)]['f1_score']:.3f}")
        print("Confusion Matrix:\n", knn_results[(metric, k)]['confusion_matrix'])


In [ ]:
# Convert results to DataFrame for plotting
results_df = pd.DataFrame(knn_results).T  # transpose so (metric, k) is index
results_df[['accuracy', 'precision', 'recall', 'f1_score']].plot(
    kind='bar', figsize=(14,6)
)
plt.title("KNN Performance Metrics for Different k and Distance Metrics")
plt.xlabel("(Distance Metric, k)")
plt.ylabel("Score")
plt.ylim(0, 1)
plt.legend(loc='lower right')
plt.show()